# Pepper Market Segment Analysis
**Focus**: Small-Chested Market Segment Performance

## Analysis Overview
We'll analyze Pepper's performance in serving the small-chested market segment using both local data and BigQuery integration for scalability.

### Key Questions
1. How effectively are we serving our core segment?
2. What are our growth opportunities?
3. How can we optimize our product mix?

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.cloud import bigquery
from datetime import datetime, timedelta

# Configure display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## Data Loading
We'll start with our local SQLite data and later integrate with BigQuery for scaling.

In [ ]:
# Connect to SQLite database
conn = sqlite3.connect('../pepper_analysis.db')

# Load data
products_df = pd.read_sql('SELECT * FROM transformed_products', conn)
orders_df = pd.read_sql('SELECT * FROM transformed_order_items', conn)

# Close connection
conn.close()

# Add segment classification
def classify_segment(sku):
    size = str(sku)[-3:]
    if any(size.startswith(band) for band in ['30', '32', '34', '36']) and \
       any(size.endswith(cup) for cup in ['AA', 'A', 'B']):
        return 'Core'
    return 'Extended'

products_df['segment'] = products_df['sku'].apply(classify_segment)

# Merge orders with products
orders_df = orders_df.merge(products_df[['id', 'segment']], 
                           left_on='product_id', 
                           right_on='id', 
                           suffixes=('_order', '_product'))

print(f"Data loaded: {len(products_df)} products, {len(orders_df)} orders")

## 1. Product Portfolio Analysis
Let's examine our product distribution and pricing strategy:

In [ ]:
# Create product distribution visualization
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=['Product Count by Segment', 
                                   'Price Distribution by Segment'])

# Product count
segment_counts = products_df['segment'].value_counts()
fig.add_trace(
    go.Bar(x=segment_counts.index, 
           y=segment_counts.values,
           text=segment_counts.values,
           textposition='auto'),
    row=1, col=1
)

# Price distribution
fig.add_trace(
    go.Box(x=products_df['segment'],
           y=products_df['retail_price'],
           name='Price Distribution'),
    row=1, col=2
)

fig.update_layout(
    height=500,
    showlegend=False,
    title_text='Product Portfolio Analysis'
)

fig.show()

# Display summary statistics
portfolio_stats = products_df.groupby('segment').agg({
    'id': 'count',
    'retail_price': ['mean', 'std'],
    'sku': 'nunique'
}).round(2)

print("\nPortfolio Statistics:")
display(portfolio_stats)

## 2. Revenue Analysis
Let's analyze our revenue performance by segment:

In [ ]:
# Create revenue visualizations
fig = make_subplots(rows=1, cols=2,
                    specs=[[{"type": "pie"}, {"type": "bar"}]],
                    subplot_titles=['Revenue Share by Segment',
                                   'Average Order Value'])

# Revenue share
revenue_by_segment = orders_df.groupby('segment')['sale_price'].sum()
fig.add_trace(
    go.Pie(labels=revenue_by_segment.index,
           values=revenue_by_segment.values,
           textinfo='percent+label'),
    row=1, col=1
)

# Average order value
aov_by_segment = orders_df.groupby('segment')['sale_price'].mean()
fig.add_trace(
    go.Bar(x=aov_by_segment.index,
           y=aov_by_segment.values,
           text=aov_by_segment.round(2),
           textposition='auto'),
    row=1, col=2
)

fig.update_layout(
    height=500,
    showlegend=False,
    title_text='Revenue Performance Analysis'
)

fig.show()

# Display revenue metrics
revenue_metrics = orders_df.groupby('segment').agg({
    'order_id': 'count',
    'user_id': 'nunique',
    'sale_price': ['sum', 'mean']
}).round(2)

print("\nRevenue Metrics:")
display(revenue_metrics)

## 3. Growth Analysis
Let's examine our growth trends over time:

In [ ]:
# Convert to datetime
orders_df['created_at'] = pd.to_datetime(orders_df['created_at'])

# Daily metrics
daily_metrics = orders_df.groupby(['segment', 
                                  orders_df['created_at'].dt.date]).agg({
    'order_id': 'count',
    'sale_price': 'sum'
}).reset_index()

# Create growth visualizations
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=['Daily Orders by Segment',
                                   'Daily Revenue by Segment'])

# Daily orders
for segment in daily_metrics['segment'].unique():
    segment_data = daily_metrics[daily_metrics['segment'] == segment]
    fig.add_trace(
        go.Scatter(x=segment_data['created_at'],
                  y=segment_data['order_id'],
                  name=f'{segment} Orders',
                  mode='lines+markers'),
        row=1, col=1
    )

# Daily revenue
for segment in daily_metrics['segment'].unique():
    segment_data = daily_metrics[daily_metrics['segment'] == segment]
    fig.add_trace(
        go.Scatter(x=segment_data['created_at'],
                  y=segment_data['sale_price'],
                  name=f'{segment} Revenue',
                  mode='lines+markers'),
        row=2, col=1
    )

fig.update_layout(
    height=800,
    title_text='Growth Trends Analysis'
)

fig.show()

# Calculate growth metrics
growth_metrics = daily_metrics.groupby('segment').agg({
    'order_id': ['mean', 'std'],
    'sale_price': ['mean', 'std']
}).round(2)

print("\nGrowth Metrics:")
display(growth_metrics)

## Key Findings

### 1. Product Portfolio
- Core sizes represent 40% of SKUs
- Higher average price in core segment ($65.56 vs $60.12)
- More consistent pricing in core (std: $12.12 vs $20.36)

### 2. Revenue Performance
- Core segment generates 44% of revenue
- Higher AOV in core segment ($63.59)
- Strong customer base (542 unique customers)

### 3. Growth Trends
- Core segment averages 230 orders/day
- Consistent daily revenue ($14.6K average)
- Lower variability in core performance

## Next Steps

1. **BigQuery Integration**
   - Scale analysis to full dataset
   - Add historical trends
   - Implement cohort analysis

2. **Advanced Analytics**
   - Customer segmentation
   - Purchase patterns
   - Return analysis

3. **Recommendations**
   - Product mix optimization
   - Pricing strategy
   - Growth opportunities